In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
raw_data_dir = "/data/put_data/cmchang/gynecology/raw_data/"
process_data_dir = "/data/put_data/cmchang/gynecology/data/"

In [4]:
dx = pd.read_csv(os.path.join(raw_data_dir, 'data_20180911.csv'))
dy = pd.read_csv(os.path.join(raw_data_dir, 'interpretation_new_180927.csv'))

print("dx.shape = %s, dy.shape = %s" % (dx.shape, dy.shape))

dx.shape = (128760, 68), dy.shape = (213, 12)


In [5]:
dx.head()

,counter,machine_type,type,bed,mins_diff,datetime,date,time,_00,_01,...,_50,_51,_52,_53,_54,_55,_56,_57,_58,_59
0,1,1,1,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,46,...,24,22,24,22,21,22,20,20,20,22
1,2,1,2,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,138,...,141,140,141,140,140,141,140,138,139,139
2,3,1,3,01,2025496,2011-04-07 14:16:00,2011-04-07,14:16:00,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,1,1,01,2025497,2011-04-07 14:17:00,2011-04-07,14:17:00,20,20,...,22,20,0,21,21,21,22,23,24,25
4,5,1,2,01,2025497,2011-04-07 14:17:00,2011-04-07,14:17:00,138,138,...,140,139,0,138,137,137,138,138,138,138


In [6]:
dy.head()

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,0,0
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,2,1
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,2,1
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,2,1
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,2,1


#### remove X but without Y

In [7]:
selector = [k in set(dy.date) for k in dx.date]

dx = dx[selector]

print("dx.shape = %s, dy.shape = %s" % (dx.shape, dy.shape))

dx.shape = (62380, 68), dy.shape = (213, 12)


#### create key

In [21]:
dx['seconds'] = (pd.to_datetime(dx.datetime) - pd.to_datetime(dx.date)).dt.seconds
dx['key'] = dx.apply(lambda row: str(row['date']) +' '+ str(row['seconds']), axis=1)

In [23]:
dy['datetime'] = dy.date + ' ' + dy.start
dy['seconds'] = (pd.to_datetime(dy.datetime) - pd.to_datetime(dy.date)).dt.seconds
dy['key'] = dy.apply(lambda row: str(row['date']) +' '+ str(row['seconds']), axis=1)

#### remove Y but without X

In [32]:
selector = [k in set(dx.key) for k in dy.key]

dy = dy[selector]

In [33]:
if set(dy.key) - set(dx.key):
    print('%i labels have no inputs.' % len(set(dy.key) - set(dx.key)))
    print(set(dy.key) - set(dx.key))
else:
    print('all labels have corresponding inputs.')


all labels have corresponding inputs.


In [34]:
dy.head()

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management,datetime,seconds,key
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,0,0,2013-01-15 17:20,62400,2013-01-15 62400
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,2,1,2013-01-15 17:54,64440,2013-01-15 64440
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,2,1,2013-01-15 18:15,65700,2013-01-15 65700
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,2,1,2013-01-15 18:29,66540,2013-01-15 66540
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,2,1,2013-01-15 18:39,67140,2013-01-15 67140


In [38]:
# only have 8 minutes ... (?)
dy[dy.key=='2015-08-28 66420']

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management,datetime,seconds,key
134,324546,LDR1,2015-08-28,18:27,18:35,135,0,1,90.0,60.0,0,0,2015-08-28 18:27,66420,2015-08-28 66420


In [39]:
dy.drop(dy.index[list(dy.key).index('2015-08-28 66420')], inplace=True)

In [40]:
dy.sort_values('key', inplace=True)

In [41]:
dy.reset_index(inplace=True)
dy.drop('index', axis=1, inplace=True)

In [42]:
dy.head()

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,deceleration,management,datetime,seconds,key
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,0,0,2013-01-15 17:20,62400,2013-01-15 62400
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,2,1,2013-01-15 17:54,64440,2013-01-15 64440
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,2,1,2013-01-15 18:15,65700,2013-01-15 65700
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,2,1,2013-01-15 18:29,66540,2013-01-15 66540
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,2,1,2013-01-15 18:39,67140,2013-01-15 67140


In [44]:
mm_dx = dx[dx.type==1]
bb_dx = dx[dx.type==2]

In [45]:
mm_dx.head(3)

,counter,machine_type,type,bed,mins_diff,datetime,date,time,_00,_01,...,_52,_53,_54,_55,_56,_57,_58,_59,seconds,key
2913,2914,1,1,LDR2,2960152,2013-01-15 15:52:00,2013-01-15,15:52:00,20,20,...,20,20,20,20,20,20,20,20,57120,2013-01-15 57120
2914,2915,1,1,LDR2,2960153,2013-01-15 15:53:00,2013-01-15,15:53:00,20,20,...,21,21,21,21,21,21,21,21,57180,2013-01-15 57180
2915,2916,1,1,LDR2,2960154,2013-01-15 15:54:00,2013-01-15,15:54:00,21,21,...,21,21,21,21,21,21,21,21,57240,2013-01-15 57240


In [49]:
mm_val_idx = list()
for k in mm_dx.columns:
    if len(k) == 3 and '_' in k:
        mm_val_idx.append(k)
        print(k, end='\t')

_00	_01	_02	_03	_04	_05	_06	_07	_08	_09	_10	_11	_12	_13	_14	_15	_16	_17	_18	_19	_20	_21	_22	_23	_24	_25	_26	_27	_28	_29	_30	_31	_32	_33	_34	_35	_36	_37	_38	_39	_40	_41	_42	_43	_44	_45	_46	_47	_48	_49	_50	_51	_52	_53	_54	_55	_56	_57	_58	_59	

In [54]:
bb_val_idx = list()
for k in bb_dx.columns:
    if len(k) == 3 and '_' in k:
        bb_val_idx.append(k)
        print(k, end='\t')

_00	_01	_02	_03	_04	_05	_06	_07	_08	_09	_10	_11	_12	_13	_14	_15	_16	_17	_18	_19	_20	_21	_22	_23	_24	_25	_26	_27	_28	_29	_30	_31	_32	_33	_34	_35	_36	_37	_38	_39	_40	_41	_42	_43	_44	_45	_46	_47	_48	_49	_50	_51	_52	_53	_54	_55	_56	_57	_58	_59	

In [56]:
dy.key

0      2013-01-15 62400
1      2013-01-15 64440
2      2013-01-15 65700
3      2013-01-15 66540
4      2013-01-15 67140
5      2013-01-15 69540
6      2013-01-15 71820
7      2013-01-15 73140
8      2013-01-15 74940
9      2013-01-15 76140
10     2013-01-15 77340
11     2013-01-18 30840
12     2013-01-18 51240
13     2013-01-28 11580
14     2013-01-28 19380
15     2013-02-15 54180
16     2013-02-21 27480
17       2013-03-03 960
18     2013-03-06 76260
19     2013-04-17 63840
20     2013-05-01 55980
21     2013-05-01 58980
22     2013-05-01 63180
23     2013-05-14 61740
24     2013-05-19 76980
25     2013-05-20 33120
26     2013-05-25 37380
27     2013-05-25 48180
28     2013-05-27 47580
29     2013-05-27 47580
             ...       
177    2017-05-07 79740
178    2017-05-13 16740
179    2017-05-13 22140
180    2017-05-13 29340
181    2017-06-10 46320
182    2017-08-16 69720
183    2017-08-23 55140
184    2017-09-12 59280
185    2017-09-12 81300
186    2017-09-13 15360
187     2017-09-

In [66]:
mm_dx

,counter,machine_type,type,bed,mins_diff,datetime,date,time,_00,_01,...,_52,_53,_54,_55,_56,_57,_58,_59,seconds,key
2913,2914,1,1,LDR2,2960152,2013-01-15 15:52:00,2013-01-15,15:52:00,20,20,...,20,20,20,20,20,20,20,20,57120,2013-01-15 57120
2914,2915,1,1,LDR2,2960153,2013-01-15 15:53:00,2013-01-15,15:53:00,20,20,...,21,21,21,21,21,21,21,21,57180,2013-01-15 57180
2915,2916,1,1,LDR2,2960154,2013-01-15 15:54:00,2013-01-15,15:54:00,21,21,...,21,21,21,21,21,21,21,21,57240,2013-01-15 57240
2916,2917,1,1,LDR2,2960155,2013-01-15 15:55:00,2013-01-15,15:55:00,21,21,...,21,21,21,21,21,20,21,21,57300,2013-01-15 57300
2917,2918,1,1,LDR1,2960224,2013-01-15 17:04:00,2013-01-15,17:04:00,0,0,...,54,56,54,56,62,61,59,55,61440,2013-01-15 61440
2919,2920,1,1,LDR1,2960225,2013-01-15 17:05:00,2013-01-15,17:05:00,0,52,...,19,0,19,19,19,19,20,20,61500,2013-01-15 61500
2921,2922,1,1,LDR1,2960226,2013-01-15 17:06:00,2013-01-15,17:06:00,19,20,...,17,17,17,19,19,0,18,18,61560,2013-01-15 61560
2923,2924,1,1,LDR1,2960227,2013-01-15 17:07:00,2013-01-15,17:07:00,18,18,...,16,16,16,15,16,16,16,16,61620,2013-01-15 61620
2925,2926,1,1,LDR1,2960228,2013-01-15 17:08:00,2013-01-15,17:08:00,16,0,...,16,16,16,16,16,16,17,17,61680,2013-01-15 61680
2927,2928,1,1,LDR1,2960229,2013-01-15 17:09:00,2013-01-15,17:09:00,16,16,...,22,0,23,23,23,22,23,22,61740,2013-01-15 61740


In [76]:
mm_dx.key

2913      2013-01-15 57120
2914      2013-01-15 57180
2915      2013-01-15 57240
2916      2013-01-15 57300
2917      2013-01-15 61440
2919      2013-01-15 61500
2921      2013-01-15 61560
2923      2013-01-15 61620
2925      2013-01-15 61680
2927      2013-01-15 61740
2929      2013-01-15 61800
2931      2013-01-15 61860
2933      2013-01-15 61920
2935      2013-01-15 61980
2937      2013-01-15 62040
2939      2013-01-15 62100
2941      2013-01-15 62160
2943      2013-01-15 62220
2945      2013-01-15 62280
2947      2013-01-15 62340
2949      2013-01-15 62400
2951      2013-01-15 62460
2953      2013-01-15 62520
2955      2013-01-15 62580
2957      2013-01-15 62640
2959      2013-01-15 62700
2961      2013-01-15 62760
2963      2013-01-15 62820
2965      2013-01-15 62880
2967      2013-01-15 62940
                ...       
128141    2018-06-29 44760
128143    2018-06-29 44820
128145    2018-06-29 44880
128147    2018-06-29 44940
128149    2018-06-29 45000
128151    2018-06-29 45060
1

In [89]:
M_list = list()
for k in dy.key:
    try:
        i = list(mm_dx.key).index(k) # find the position number

        tmp = mm_dx[i:(i+10)]
        checkpoint = tmp.seconds.tolist()
        if (checkpoint[-1] - checkpoint[0]) == 60*9:
            M_list.append(np.array(mm_dx[i:(i+10)][mm_val_idx]).reshape(-1))
            continue
        else:
            print('%s has problem' % k)
    except ValueError as err:
        print(err)

In [90]:
B_list = list()
for k in dy.key:
    try:
        i = list(bb_dx.key).index(k)
        tmp = bb_dx[i:(i+10)]
        checkpoint = tmp.seconds.tolist()
        if (checkpoint[-1] - checkpoint[0]) == 60*9:
            B_list.append(np.array(bb_dx[i:(i+10)][bb_val_idx]).reshape(-1))
            continue
        else:
            print('%s has problem' % k)
    except ValueError as err:
        print(err)

In [103]:
M = pd.DataFrame(M_list, columns=['m-{0:0>3}'.format(i) for i in range(len(M_list[0]))])

B = pd.DataFrame(B_list, columns=['b-{0:0>3}'.format(i) for i in range(len(B_list[0]))])

In [26]:
B.shape

(207, 600)

In [27]:
M.shape

(207, 600)

In [105]:
dy.shape

(207, 15)

In [108]:
dM = pd.concat([dy, M], axis=1)

#dM.to_csv(os.path.join(process_data_dir, 'data_M.csv'), index=False)

In [110]:
dB = pd.concat([dy, B], axis=1)

#dB.to_csv(os.path.join(process_data_dir, 'data_B.csv'), index=False)

In [112]:
df = pd.concat([dy, M], axis=1)
df = pd.concat([df, B], axis=1)

#df.to_csv(os.path.join(process_data_dir, 'data_merged.csv'), index=False)

In [113]:
df

,ID,bed,date,start,end,FHB,variability,UA,UA_duration,UA_interval,...,b-590,b-591,b-592,b-593,b-594,b-595,b-596,b-597,b-598,b-599
0,288427,LDR1,2013-01-15,17:20,17:30,140,0,0,0.0,NaN,...,140,140,140,0,142,147,149,152,154,148
1,288427,LDR1,2013-01-15,17:54,18:04,140,0,1,60.0,120.0,...,0,147,150,157,157,0,151,150,150,148
2,288427,LDR1,2013-01-15,18:15,18:25,155,0,0,0.0,NaN,...,142,146,149,0,150,146,141,140,0,140
3,288427,LDR1,2013-01-15,18:29,18:39,150,0,0,0.0,NaN,...,150,150,151,154,154,154,154,155,156,157
4,288427,LDR1,2013-01-15,18:39,18:49,155,0,1,120.0,60.0,...,160,0,160,158,157,154,0,154,153,152
5,288427,LDR1,2013-01-15,19:19,19:29,160,0,1,90.0,90.0,...,162,162,161,160,0,161,161,162,0,164
6,288427,LDR1,2013-01-15,19:57,20:07,155,1,1,60.0,60.0,...,154,156,158,159,160,158,146,144,149,154
7,288427,LDR1,2013-01-15,20:19,20:29,155,0,1,90.0,60.0,...,157,0,159,160,162,163,161,159,158,158
8,288427,LDR1,2013-01-15,20:49,20:59,155,0,1,90.0,60.0,...,167,167,165,167,168,169,0,171,172,174
9,288427,LDR1,2013-01-15,21:09,21:19,160,0,1,90.0,60.0,...,164,165,155,156,160,163,165,166,0,168
